In [15]:
import pickle
from docuverse.utils import open_stream
cache_file="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora____en__corpus.small.jsonl_512_100_True_all_gte-small.pickle.xz"
import json
data=pickle.load(open_stream(cache_file))
questions = [
    "How can I get free gems in Clash of Clans?",
    "How can I get free gems Clash of Clans?",
    "How do you feel when someone upvotes your answer on Quora?",
    "What are the best thriller movie in Hollywood?",
    "What should someone do to overcome anxiety?"
]

In [4]:
MODEL = ("/home/raduf/sandbox2/docuverse/models/slate.30m.english.rtrvr-20240719T181101")
data_list = []
from docuverse.utils.embeddings.dense_embedding_function import DenseEmbeddingFunction
model = DenseEmbeddingFunction(MODEL)

=== done initializing model


In [5]:
embeddings = model.encode([d['text'] for d in data], show_progress_bar=True)
keys_to_keep = {"text"}
data_list = [{**{k:v for k, v in d.items() if k in keys_to_keep}, '_id': d['id'], 'qembedding':embeddings[i]} for i,d in enumerate(data)]

Batches:   0%|          | 0/301 [00:00<?, ?it/s]

In [7]:
data_list[0].keys()

dict_keys(['text', '_id', 'qembedding'])

In [32]:

from pymilvus import (
    MilvusClient,
    DataType
)

def test_search(vectors, vector_for_query=None, metric="IP"):
    truncate_dim = 384
    collection_name = "test"
    vector_field_name = "qembedding"

    client = MilvusClient("test.db")
    schema = client.create_schema(auto_id=True, enable_dynamic_field=True, primary_field="id")

    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=50000)
    schema.add_field(field_name="_id", datatype=DataType.VARCHAR, max_length=50000)
    schema.add_field(field_name=vector_field_name, datatype=DataType.FLOAT_VECTOR, dim=truncate_dim)

    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name=vector_field_name,
        index_type="FLAT",
        metric_type=metric,
        params={"nlist": 1024}
    )

    if client.has_collection(collection_name=collection_name):
        client.drop_collection(collection_name=collection_name)
    client.create_collection(
        collection_name=collection_name, schema=schema, index_params=index_params
    )
    if vector_for_query is None:
        entities = vectors
        test = vectors[0:3]
    else:
        entities = vectors
        if isinstance(vector_for_query, list) and isinstance(vector_for_query[0], dict):
            test = vector_for_query
        else:
            test = [{vector_field_name: e} for e in vector_for_query]
    print(test[0].keys())
    insert_result = client.insert(collection_name=collection_name, data=entities)
    # print(insert_result)
    return client.search(
        collection_name=collection_name,
        # data=[t[vector_field_name] for t in test],
        data=test,
        search_params={"metric_type": metric, "params": {"nprobe": 10}},
        anns_field=vector_field_name,
        limit=10,
        output_fields=["text", "_id"],
    )

def test_search1(vectors, vector_for_query=None, metric="IP"):
    truncate_dim = 4
    collection_name = "test"
    vector_field_name = "qembedding"

    client = MilvusClient("test.db")
    schema = client.create_schema(auto_id=True, enable_dynamic_field=True, primary_field="id")

    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=50000)
    schema.add_field(field_name="_id", datatype=DataType.VARCHAR, max_length=50000)
    schema.add_field(field_name=vector_field_name, datatype=DataType.FLOAT_VECTOR, dim=truncate_dim)

    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name=vector_field_name,
        index_type="FLAT",
        metric_type=metric,
        params={"nlist": 1024}
    )

    if client.has_collection(collection_name=collection_name):
        client.drop_collection(collection_name=collection_name)
    client.create_collection(
        collection_name=collection_name, schema=schema, index_params=index_params
    )
    if vector_for_query is None:
        entities = vectors[:-2]
        test = vectors[-2:]
    else:
        entities = vectors
        test = vector_for_query
    # entities = [
    #     {"source": "hello", vector_field_name: vector_for_db}
    # ]
    insert_result = client.insert(collection_name=collection_name, data=entities)
    print(insert_result)
    return client.search(
        collection_name=collection_name,
        data=test,
        search_params={"metric_type": metric, "params": {"nprobe": 10}},
        anns_field=vector_field_name,
        limit=10,
        output_fields=["source"],
    )


In [30]:
def test_search(vectors, vector_for_query=None, metric="IP"):
    truncate_dim = 384
    collection_name = "test"
    vector_field_name = "qembedding"

    client = MilvusClient("test.db")
    schema = client.create_schema(auto_id=True, enable_dynamic_field=True, primary_field="id")

    schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
    schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=50000)
    schema.add_field(field_name="_id", datatype=DataType.VARCHAR, max_length=50000)
    schema.add_field(field_name=vector_field_name, datatype=DataType.FLOAT_VECTOR, dim=truncate_dim)

    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name=vector_field_name,
        index_type="FLAT",
        metric_type=metric,
        params={"nlist": 1024}
    )

    if client.has_collection(collection_name=collection_name):
        client.drop_collection(collection_name=collection_name)
    client.create_collection(
        collection_name=collection_name, schema=schema, index_params=index_params
    )
    if vector_for_query is None:
        entities = vectors
        test = vectors[0:3]
    else:
        entities = vectors
        if isinstance(vector_for_query, list) and isinstance(vector_for_query[0], dict):
            test = vector_for_query
        else:
            test = [{vector_field_name: e} for e in vector_for_query]
    print(test[0].keys())
    insert_result = client.insert(collection_name=collection_name, data=entities)
    # print(insert_result)
    return client.search(
        collection_name=collection_name,
        # data=[t[vector_field_name] for t in test],
        data=test,
        search_params={"metric_type": metric, "params": {"nprobe": 10}},
        anns_field=vector_field_name,
        limit=10,
        output_fields=["text", "_id"],
    )
    
questions=[d['text'] for d in data_list[0:10]]
answers=test_search(data_list[:20], questions)


RPC error: [search], <ParamError: (code=1, message=`search_data` value [{'qembedding': "I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?"}, {'qembedding': "I can't remember my Gmail password or my recovery email. How can I recover my e-mail?"}, {'qembedding': 'What is a narcissistic personality disorder?'}, {'qembedding': 'What is narcissistic personality disorder?'}, {'qembedding': 'What is the greatest mystery of all time?'}, {'qembedding': 'What does the Quora website look like to members of Quora moderation?'}, {'qembedding': 'Is it possible to pursue many different things in life?'}, {'qembedding': 'No fanboys please, but who was the true batman, Christian Bale or Ben Affleck?'}, {'qembedding': 'What will be the impact of scrapping of ₹500 and ₹1000 rupee notes on the real estate market?'}, {'qembedding': 'What is the easiest way to become a billionaire($)?'}] is illegal)>, <Time:{'RPC start

ParamError: <ParamError: (code=1, message=`search_data` value [{'qembedding': "I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?"}, {'qembedding': "I can't remember my Gmail password or my recovery email. How can I recover my e-mail?"}, {'qembedding': 'What is a narcissistic personality disorder?'}, {'qembedding': 'What is narcissistic personality disorder?'}, {'qembedding': 'What is the greatest mystery of all time?'}, {'qembedding': 'What does the Quora website look like to members of Quora moderation?'}, {'qembedding': 'Is it possible to pursue many different things in life?'}, {'qembedding': 'No fanboys please, but who was the true batman, Christian Bale or Ben Affleck?'}, {'qembedding': 'What will be the impact of scrapping of ₹500 and ₹1000 rupee notes on the real estate market?'}, {'qembedding': 'What is the easiest way to become a billionaire($)?'}] is illegal)>

In [24]:
for q, res in zip(questions, answers):
    print(f"Question: {q['text']}")
    if len(res) == 0:
        print("  ** No results found. **")
    else:
        for r in res:
            print({'id': r['entity']['_id'], 'answer': r['entity']['text'], 'score': r['distance']})
        print("\n")

Question: I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?
{'id': '117-0-139', 'answer': "I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?", 'score': 1.000000238418579}
{'id': '106577-0-51', 'answer': 'What is the best music to listen to while studying?', 'score': 0.9982950091362}
{'id': '348001-0-30', 'answer': 'What is New Zealand known for?', 'score': 0.7475979328155518}
{'id': '383079-0-123', 'answer': 'How many days will it take to dispatch pan card after application has received at income tax office for further processing?', 'score': 0.741649866104126}
{'id': '124606-0-30', 'answer': 'How do I play cards by myself?', 'score': 0.7245960235595703}
{'id': '416946-0-42', 'answer': 'What are the best ways to confront a liar?', 'score': 0.6999878883361816}
{'id': '471345-0-108', 'answer': "I am a engineerin

In [23]:
questions[0]['text']

"I was suddenly logged off Gmail. I can't remember my Gmail password and just realized the recovery email is no longer alive. What can I do?"

In [37]:
client = MilvusClient("http://localhost:19530")

In [39]:
client.list_collections()

['beir_msmarco_milvus_bm25_512_100_08292024',
 'beir_hotpotqa_milvus_bm25_512_100_08292024',
 'beir_quora_new_milvus_hybrid_512_100_08292024',
 'beir_quora_small_milvus_dense_512_100_08292024',
 'huggingface_test1',
 'ibmsw_gte_small_256_50_1889300',
 'beir_dbpedia_entity_milvus_bm25_512_100_08292024',
 'beir_quora_new_milvus_dense_512_100_08292024',
 'test',
 'ibmsw_gte_small_512_30_1889300',
 'huggingface_test2',
 'nfcorpus_milvus_bm25_512_50_1889300',
 'beir_fiqa_milvus_bm25_512_100_08292024',
 'beir_quora_milvus_bm25_512_100_08292024',
 'beir_ibmsw_milvus_bm25_512_100_08292024',
 'huggingface_test',
 'ibmsw_gte_small_512_100_1889300',
 'ibmsw_gte_small_512_50_1889300',
 'beir_fever_milvus_bm25_512_100_08292024',
 'ibmsw_gte_small_512_50_09272024']

In [87]:
client.get_collection_stats("test3")

{'row_count': 0}

In [41]:
client.list_indexes("test")

['qembedding']

In [83]:
client.describe_collection("test1")

{'collection_name': 'test1',
 'auto_id': True,
 'num_shards': 1,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'text',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 50000}},
  {'field_id': 102,
   'name': '_id',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 50000}},
  {'field_id': 103,
   'name': 'qembedding',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 384}}],
 'aliases': [],
 'collection_id': 452890847312229091,
 'consistency_level': 2,
 'properties': {},
 'num_partitions': 1,
 'enable_dynamic_field': True}

In [47]:
from pymilvus import utility, connections
connections.connect(host="localhost", port='19530')

In [81]:
utility.index_building_progress("test2", "qembedding")


{'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

In [66]:
client.get_collection_stats("test")

{'row_count': 9617}

In [78]:
client.describe_index("test1", "qembedding")

{'nlist': '1024',
 'index_type': 'FLAT',
 'metric_type': 'IP',
 'field_name': 'qembedding',
 'index_name': 'qembedding',
 'total_rows': 0,
 'indexed_rows': 0,
 'pending_index_rows': 0,
 'state': 'Finished'}

In [76]:
client.drop_collection("test3")